# Parallelization

- Parallel programming is hard.
- The main challenge is multiple tasks accessing the same resource.
- With multithreading, order of execution may change in subtle ways.
- Fortunately, many computational tasks are ["embarassingly parallel"](https://en.wikipedia.org/wiki/Embarrassingly_parallel), and parallelization can provide great speedups at low cost.
- Parallelization will always multiply memory usage. Won't help if your processing is *memory bound*.
- To parallelize your code, you can use multiple **threads** or multiple **processes**.



# Multithreading

Threads are lightweight and share the same memory space.

- In Python, because of GIL (global interpreter lock), only one thread can be executed at a time.
- Parallelization is achieved by switching between threads when they idle.
- Best used with *I/O bound* tasks (CPU load under 100% is good indicator).
  


In [ ]:
import threading
import concurrent.futures
from time import time, sleep

def cpu_idle(t):
    thread = threading.current_thread().name
    print(thread, 'says hello')
    sleep(t)
    print(thread, 'says bye')
    
def cpu_burn(t):
    thread = threading.current_thread().name
    print(thread, 'says hello')
    t0 = time()
    while time() - t0 < t:
        x = 1
    print(thread, 'says bye')

In [ ]:
%%time
# sequential
times = [2] * 3
for _ in map(cpu_idle, times):
    pass

Notice the problem of writing to a shared resource from multiple threads. In this case, shared resource is the standard output stream.

In [ ]:
%%time
# multithreaded, low CPU load
times = [2] * 3
with concurrent.futures.ThreadPoolExecutor(max_workers=len(times)) as pool:
    pool.map(cpu_idle, times)

In [ ]:
%%time
# multithreaded, high CPU load
# if you see no difference, increase length of list
times = [2] * 3
with concurrent.futures.ThreadPoolExecutor(max_workers=len(times)) as pool:
    pool.map(cpu_burn, times)

## Example: download many files

This task is network I/O bound, CPU is idling while waiting for the next chunk of data to arrive.

[Cartographic boundary files](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html) - Census Bureau

> The cartographic boundary files are simplified representations of selected geographic areas from the Census Bureau’s MAF/TIGER geographic database. These boundary files are specifically designed for small scale thematic mapping.

In [ ]:
import concurrent.futures
import threading

from tools import download_file, unzip, ResourceMonitor, tracts_state_00_aa

def download_state_tracts(state_code):
    url = f'https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_{state_code}_tract_500k.zip'
    f = download_file(url, f'data/tracts/{state_code}', overwrite=True, verbose=False)
    print(threading.current_thread().name, 'finished', state_code)
    return f

### Sequential

In [ ]:
mon = ResourceMonitor()
mon.start()

for _ in map(download_state_tracts, tracts_state_00_aa):
    pass

mon.stop()
mon.plot()

### Parallel
With multiple threads CPU can go over 100%.


In [ ]:
mon = ResourceMonitor(interval=0.2)
mon.start()

with concurrent.futures.ThreadPoolExecutor() as pool:
    pool.map(download_state_tracts, tracts_state_00_aa)

mon.stop()
mon.plot()

# Multiprocessing

Multiprocessing spawns subprocesses and initial memory state is cloned to each.

- Every process then has independent memory space. Less risk of corrupting shared state.
- Because initial memory is copied to each process, memory usage is higher.
- Best used with *CPU bound* tasks.

In [ ]:
import os
import concurrent.futures
from time import time, sleep

def cpu_idle(t):
    print(os.getpid(), 'says hello')
    sleep(t)
    print(os.getpid(), 'says bye')
    
def cpu_burn(t):
    print(os.getpid(), 'says hello')
    t0 = time()
    while time() - t0 < t:
        x = 1
    print(os.getpid(), 'says bye')

In [ ]:
%%time
# multiprocessing, low CPU load
times = [2] * 3
with concurrent.futures.ProcessPoolExecutor(max_workers=len(times)) as pool:
    pool.map(cpu_idle, times)

In [ ]:
%%time
# multiprocessing, high CPU load
times = [2] * 3
with concurrent.futures.ProcessPoolExecutor(max_workers=len(times)) as pool:
    pool.map(cpu_burn, times)

## memory under multiprocessing

In [ ]:
# restart kernel
import os
import concurrent.futures
from time import time, sleep
import psutil

x = [1] * 20_000_000

def use_mem(i):
    p = psutil.Process()
    # every process gets different delay to avoid simultaneous writes to stdout
    sleep(i / 2)
    mem = p.memory_info().rss // 2**20
    print(i, os.getpid(), 'initial usage:', mem, 'MB')
    sleep(3)
    mem = p.memory_info().rss // 2**20
    print(i, os.getpid(), 'usage:', mem, 'MB')
    # make a copy of x
    y = list(x)
    sleep(1)
    mem = p.memory_info().rss // 2**20
    print(i, os.getpid(), 'final usage:', mem, 'MB')

In [ ]:
%%time
n = 2
with concurrent.futures.ProcessPoolExecutor(n) as pool:
    pool.map(use_mem, range(n))

## Example: identify census tracts from coordinates

For every establishment, we want to identify census tract that it belongs to. This requires to perform "point in shape" computation, CPU intensive task, many times. Memory usage is not high, and so we will parallelize the task over multiple processes.

In [ ]:
import concurrent.futures

import pandas as pd
import geopandas as gpd

from tools import download_file, unzip, tracts_state_00_aa, tracts_state_aa_00

### Unzip downloaded shapefiles

Files are small, so this is very fast to do sequentially. But this task could also be parallelized.

In [ ]:
def unzip_tract(state_code):
    f = f'data/tracts/{state_code}/cb_2019_{state_code}_tract_500k.zip'
    unzip(f, f'data/tracts/{state_code}', overwrite=True, verbose=False)

for sc in tracts_state_00_aa:
    unzip_tract(sc)

### Identify tracts

Perform "spatial join" of all establishment coordinats against all tract shapes.

In [ ]:
def tracts_from_coords(state):
    state_code = tracts_state_aa_00[state]
    df = pd.read_parquet('data/synig.pq', columns=['ABI', 'LONGITUDE', 'LATITUDE'],
                         filters=[('YEAR', '==', 2001), ('STATE', '==', state)])
    if len(df) == 0:
        return
    df = gpd.GeoDataFrame(df)
    df['LONLAT'] = gpd.points_from_xy(df['LONGITUDE'], df['LATITUDE'])
    df = df.set_geometry('LONLAT', crs={'init': 'epsg:4326'})
    tracts = gpd.read_file(f'data/tracts/{state_code}/cb_2019_{state_code}_tract_500k.shp')
    tracts = tracts[['GEOID', 'geometry']].to_crs({'init': 'epsg:4326'})
    df = gpd.sjoin(df, tracts, 'left', 'within')
    return df[['ABI', 'GEOID']]

In [ ]:
%%time
# sequential execution is rather slow
# short list of states for demonstration
states = list(tracts_state_aa_00.keys())[:4]

df = []
for state in states:
    print(state, end=' ')
    df.append(tracts_from_coords(state))
print()

df = pd.concat([x for x in df if x is not None], ignore_index=True)

In [ ]:
%%time
# parallel execution
# short list of states for demonstration
states = list(tracts_state_aa_00.keys())[:4]

with concurrent.futures.ProcessPoolExecutor(4) as pool:
    df = pool.map(tracts_from_coords, states)
df = pd.concat([x for x in df if x is not None], ignore_index=True)